#Regularization for a classifier of the cat/dog data
##Plot functions for training evaluation
These function will provide performance curves for each epoch of the training process. This will allow us later to judge if overfitting occured or not and if our regularization approaches were successful.

In [19]:
import matplotlib.pyplot as plt

def plot_histories(histories, keys=['loss', 'acc']):
    fig, axs = plt.subplots(len(keys), 1, figsize=(20,16))
    #fig.figsize=(20,16)
    if len(keys) == 1:
        axs = [axs]
    # one plot for each specified key
    for i, ax in enumerate(axs):
        key = keys[i]
        for name, history in histories:
            val = ax.plot(history.epoch, history.history['val_'+key],
                           '--', label=name.title()+' Val')
            ax.plot(history.epoch, history.history[key], color=val[0].get_color(),
                    label=name.title()+' Train')

        ax.set_xlabel('Epochs')
        ax.set_ylabel(key.replace('_',' ').title())
        ax.legend()
        ax.set_title(key)
        ax.set_xlim([0,max(history.epoch)])
    plt.show()

##Setting input paths


In [20]:
import numpy as np

In [21]:
import os

os.getcwd()

'/content'

In [22]:
# TODO: maybe have to adjust the base_dir if IO error occurs
base_dir = os.path.join('..','..', '..', 'data','cats-dogs')
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

train_cats_dir = os.path.join(train_dir, 'cats')
train_dogs_dir = os.path.join(train_dir, 'dogs')

validation_cats_dir = os.path.join(validation_dir, 'cats')
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

In [23]:
print('total training cat images:', len(os.listdir(train_cats_dir)))
print('total validation dog images:', len(os.listdir(validation_dogs_dir)))

FileNotFoundError: ignored

##Setting up the input pipeline


In [24]:
# this is the augmentation configuration we will use for training
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
        #shear_range=0.2,
        #zoom_range=0.2,
        #horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

In [25]:
batch_size = 64

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        train_dir,  # this is the target directory
        target_size=(150, 150),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')

FileNotFoundError: ignored

##The baseline model
Our first model whose performance we will try to improve by means of regularization. We will see that it is powerful enough to classify the training data perfectly. But this is of course not our main goal. So maybe it is already to powerful (in terms of parameters)?

In [26]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

def get_baseline_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    
    return model

In [27]:
m_base = get_baseline_model()
#m_base = get_baseline_model_cnn()
print(m_base.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 148, 148, 32)      896       
                                                                 
 activation_10 (Activation)  (None, 148, 148, 32)      0         
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 74, 74, 32)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 72, 72, 32)        9248      
                                                                 
 activation_11 (Activation)  (None, 72, 72, 32)        0         
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 36, 36, 32)       0         
 2D)                                                  

In [28]:
from keras import optimizers

m_base.compile(optimizer='rmsprop',
               loss="binary_crossentropy", 
               metrics=['accuracy'])

###Start training


In [29]:
epochs = 20

hist_base = m_base.fit_generator(train_generator,
                                 epochs=epochs,
                                 steps_per_epoch=2000 // batch_size,
                                 validation_data=validation_generator,
                                 validation_steps=800 // batch_size)

NameError: ignored

In [30]:
plot_histories([('baseline', hist_base)], keys=['loss'])


NameError: ignored

##Adjust model size
### smaller model
Since the baseline model is able to classify the training data almost perfectly, we will try to lessen its capacity by reducing the number of parameters, hoping to reduce the amount of overfitting.

In [31]:
def get_smaller_model():
    model = Sequential()
    
    model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # conv block with only half as many filters
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    
    return model

In [32]:
m_small = get_smaller_model()
print(m_small.summary())

m_small.compile(optimizer='rmsprop',
               loss="binary_crossentropy", 
               metrics=['accuracy'])

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 148, 148, 32)      896       
                                                                 
 activation_15 (Activation)  (None, 148, 148, 32)      0         
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 74, 74, 32)       0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 72, 72, 32)        9248      
                                                                 
 activation_16 (Activation)  (None, 72, 72, 32)        0         
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 36, 36, 32)       0         
 g2D)                                                 

In [33]:
hist_small = m_small.fit_generator(train_generator,
                                   epochs=epochs,
                                   steps_per_epoch=2000 // batch_size,
                                   validation_data=validation_generator,
                                   validation_steps=800 // batch_size)

NameError: ignored

In [34]:
plot_histories([('small', hist_small),
                ('baseline', hist_base), ], keys=['loss'])

NameError: ignored

##larger model
By making the model larger we expect to see an even more extreme form of overfitting.

In [35]:
def get_larger_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
    # twice as many fc neurons here!
    model.add(Dense(128))
    model.add(Activation('relu'))
    # and another dense layer!
    model.add(Dense(64))
    model.add(Activation('relu'))
    
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    
    return model

In [36]:
m_large = get_larger_model()
print(m_large.summary())

m_large.compile(optimizer='rmsprop',
                loss="binary_crossentropy", 
                metrics=['accuracy'])

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 148, 148, 32)      896       
                                                                 
 activation_20 (Activation)  (None, 148, 148, 32)      0         
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 74, 74, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_13 (Conv2D)          (None, 72, 72, 32)        9248      
                                                                 
 activation_21 (Activation)  (None, 72, 72, 32)        0         
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 36, 36, 32)       0         
 g2D)                                                 

In [37]:
hist_large = m_large.fit_generator(train_generator,
                                   epochs=epochs,
                                   steps_per_epoch=2000 // batch_size,
                                   validation_data=validation_generator,
                                   validation_steps=800 // batch_size)

NameError: ignored

In [38]:
plot_histories([('small', hist_small),
                ('baseline', hist_base), 
                ('large', hist_large)], keys=['loss'])

NameError: ignored

##Using "early stopping"
The idea here is to stop the training procedure before the model starts to adapt too much on the training samples. We go on as long as the validation loss drops by at least 0.1 and stop if it does not do so during 5 consecutive training epochs.

Note: This is a kind of data snooping since information of the validation set is used during training which should in general be a no-go. We show it here for demonstration only! The way to go would be to divide your data into training, validation and test set and do the early stopping based on the loss on the validation set. After the model is then trained, you should validate it on the test set which has not been looked at before.

In [39]:
from keras.callbacks import EarlyStopping

m_base = get_baseline_model()

m_base.compile(optimizer="rmsprop",
               #optimizers.Adadelta(),
               loss="binary_crossentropy", 
               metrics=['accuracy'])

hist_earlystop = m_base.fit_generator(train_generator,
                                      epochs=epochs,
                                      steps_per_epoch=2000 // batch_size,
                                      validation_data=validation_generator,
                                      validation_steps=800 // batch_size,
                                      callbacks=[
                                          EarlyStopping(monitor='val_loss', min_delta=0.1, patience=5, verbose=1)])


NameError: ignored

In [ ]:
plot_histories([('early_stopping', hist_earlystop), 
                ('baseline', hist_base)], keys=['loss'])

##Weight regularisation
You may be familiar with Occam's Razor principle: given two explanations for something, the explanation most likely to be correct is the "simplest" one, the one that makes the least amount of assumptions. This also applies to the models learned by neural networks: given some training data and a network architecture, there are multiple sets of weights values (multiple models) that could explain the data, and simpler models are less likely to overfit than complex ones.

A "simple model" in this context is a model where the distribution of parameter values has less entropy (or a model with fewer parameters altogether, as we saw in the section above). Thus a common way to mitigate overfitting is to put constraints on the complexity of a network by forcing its weights only to take small values, which makes the distribution of weight values more "regular". This is called "weight regularization", and it is done by adding to the loss function of the network a cost associated with having large weights. This cost comes in two flavors:

* L1 regularization, where the cost added is proportional to the absolute value of the weights coefficients (i.e. to what is called the "L1 norm" of the weights).

* L2 regularization, where the cost added is proportional to the square of the value of the weights coefficients (i.e. to what is called the "L2 norm" of the weights). L2 regularization is also called weight decay in the context of neural networks. Don't let the different name confuse you: weight decay is mathematically the exact same as L2 regularization.

In Keras, weight regularization is added by passing weight regularizer instances to layers as keyword arguments. Let's try both flavors now.

###Using L1 regularization


In [ ]:
# copy-paste the baseline model, but add regularization terms to each layer
from keras import regularizers

reg_weight = 1.e-4

def get_l1_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), kernel_regularizer=regularizers.l1(reg_weight), input_shape=(150, 150, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, (3, 3), kernel_regularizer=regularizers.l1(reg_weight)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3), kernel_regularizer=regularizers.l1(reg_weight)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
    model.add(Dense(64, kernel_regularizer=regularizers.l1(reg_weight)))
    model.add(Activation('relu'))
    model.add(Dense(1, kernel_regularizer=regularizers.l1(reg_weight)))
    model.add(Activation('sigmoid'))
    
    return model

l1(0.001) means that every coefficient in the weight matrix of the layer will add 0.001 * abs(weight_coefficient_value) to the total loss of the network.

In [40]:
m_l1 = get_l1_model()

m_l1.compile(optimizer="rmsprop",
             loss="binary_crossentropy", 
             metrics=['accuracy'])

hist_l1 = m_l1.fit_generator(train_generator,
                                 epochs=epochs,
                                 steps_per_epoch=2000 // batch_size,
                                 validation_data=validation_generator,
                                 validation_steps=800 // batch_size)

NameError: ignored

In [ ]:
plot_histories([('l1_reg', hist_l1), 
                ('baseline', hist_base)], keys=['loss'])

Note that because this penalty is only added at training time, the loss for this network will be higher at training than at test time.

###Using L2 regularization

In [41]:
# copy-paste the baseline model, but add regularization terms to each layer
from keras import regularizers

reg_weight = 1.e-4

def get_l2_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(reg_weight), input_shape=(150, 150, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(reg_weight)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(reg_weight)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
    model.add(Dense(64, kernel_regularizer=regularizers.l2(reg_weight)))
    model.add(Activation('relu'))
    model.add(Dense(1, kernel_regularizer=regularizers.l2(reg_weight)))
    model.add(Activation('sigmoid'))
    return model

In [42]:
m_l2 = get_l2_model()

m_l2.compile(optimizer="rmsprop",
             loss="binary_crossentropy", 
             metrics=['accuracy'])

hist_l2 = m_l2.fit_generator(train_generator,
                             epochs=epochs,
                             steps_per_epoch=2000 // batch_size,
                             validation_data=validation_generator,
                             validation_steps=800 // batch_size)

NameError: ignored

In [43]:
plot_histories([('l2_reg', hist_l2), 
                ('baseline', hist_base)], keys=['loss'])

NameError: ignored

In [ ]:
plot_histories([('l1_reg', hist_l1), 
                ('baseline', hist_base),
                ('l2_reg', hist_l2)], keys=['loss'])

##Using L1 and L2 regularization


In [44]:
reg_weight = 1.e-4

def get_l1_l2_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), kernel_regularizer=regularizers.l1_l2(reg_weight, reg_weight), 
                     input_shape=(150, 150, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, (3, 3), kernel_regularizer=regularizers.l1_l2(reg_weight, reg_weight)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3), kernel_regularizer=regularizers.l1_l2(reg_weight, reg_weight)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
    model.add(Dense(64, kernel_regularizer=regularizers.l1_l2(reg_weight, reg_weight)))
    model.add(Activation('relu'))
    model.add(Dense(1, kernel_regularizer=regularizers.l1_l2(reg_weight, reg_weight)))
    model.add(Activation('sigmoid'))

    
    return model

In [45]:
m_l1l2 = get_l1_l2_model()

m_l1l2.compile(optimizer="rmsprop",
               loss="binary_crossentropy", 
               metrics=['accuracy'])

hist_l1l2 = m_l1l2.fit_generator(train_generator,
                                 epochs=epochs,
                                 steps_per_epoch=2000 // batch_size,
                                 validation_data=validation_generator,
                                 validation_steps=800 // batch_size)

NameError: ignored

In [46]:
plot_histories([('l1_l2_reg', hist_l1l2), 
                ('baseline', hist_base)], keys=['loss'])

NameError: ignored

In [ ]:
plot_histories([('l1_reg', hist_l1), 
                ('baseline', hist_base),
                ('l2_reg', hist_l2),
                ('l1_l2_reg', hist_l1l2)], keys=['loss'])

##Using dropout
Dropout is one of the most commonly used regularization techniques for neural networks, developed by Hinton and his students at the University of Toronto. Dropout, applied to a layer, consists of randomly "dropping out" (i.e. set to zero) a number of output features of the layer during training. Let's say a given layer would normally have returned a vector [0.2, 0.5, 1.3, 0.8, 1.1] for a given input sample during training; after applying dropout, this vector will have a few zero entries distributed at random, e.g. [0, 0.5, 1.3, 0, 1.1]. The "dropout rate" is the fraction of the features that are being zeroed-out; it is usually set between 0.2 and 0.5. At test time, no units are dropped out, and instead the layer's output values are scaled down by a factor equal to the dropout rate, so as to balance for the fact that more units are active than at training time.

In tf.keras you can introduce dropout in a network via the Dropout layer, which gets applied to the output of layer right before.

In [47]:
from keras.layers import Dropout

def get_dropout_model(rate=0.5):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(rate))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    
    return model

In [48]:
m_drop = get_dropout_model()

m_drop.compile(optimizer="rmsprop",
               loss="binary_crossentropy", 
               metrics=['accuracy'])

hist_drop = m_drop.fit_generator(train_generator,
                                 epochs=epochs,
                                 steps_per_epoch=2000 // batch_size,
                                 validation_data=validation_generator,
                                 validation_steps=800 // batch_size)

NameError: ignored

In [ ]:
plot_histories([('dropout', hist_drop), 
                ('baseline', hist_base)], keys=['loss'])

Let's investigate the effect of a higher dropout rate.



In [49]:
m_more_drop = get_dropout_model(0.75)
m_more_drop.compile(optimizer="rmsprop",
                    loss="binary_crossentropy", 
                    metrics=['accuracy'])

hist_more_drop = m_more_drop.fit_generator(train_generator,
                                           epochs=epochs,
                                           steps_per_epoch=2000 // batch_size,
                                           validation_data=validation_generator,
                                           validation_steps=800 // batch_size)

NameError: ignored

In [50]:
plot_histories([('dropout', hist_drop), 
                ('baseline', hist_base),
                ('more_dropout', hist_more_drop)], keys=['loss'])

NameError: ignored

##Using batch normalization
Intuitively you want the variances of each layers input of your network to be nice. You do not want them to go to zero nor do you want them to explode. Ideally you want everything be normalized throughout your network. Batch normalization is a new technique that does this: it just takes the batch mean and standard deviation of each of our features (i.e. for each pixel across the data) and then sets its value to be such that it has zero mean and unit variance.

It turns out that this function is differentiable and we can backpropagate it! By inserting now a batch normalization layer in our network we will take in as input some x and normalize it. So for a given input activation we can apply this normalization independently across our batch. So the hope is that by doing this the network will learn faster because it will not have to adapt to changes in the distribution of the inputs it receives.

Benefits of BN
* Improves gradient flow through network
* Allow higher learning rates
* Reduce strong dependency on initialization
* Acts as a form of regularization
Note:
  * at test time BN layer works differently
  * Mean and Std not computed based on batch (since you might want to predict single images)
  * Use a running mean and std. (maintained during training)

One more subtle issue is that it actually acts as a form of regularization. The way it acts as a regularizer is that with BN, if you have some input x, and it goes through the network then its representation in the network is also influenced by all the other examples in your batch since we use the batch mean and variance. So BN is using all the images of the batch a particular image was sampled in. So it randomizes the images position in the representation space which has a nice regularizing effect.

Note that at test time the BN layer acts differently. At test time, we want it to be a deterministic function. So we need to remember our mean and variance across the dataset. We can either compute it over all our data or much easier is to have a running mean and variance while training and then use that directly.

In [51]:
from keras.layers import BatchNormalization

def get_batchnorm_model():    
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, (3, 3)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
    model.add(Dense(64))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    
    return model

In [52]:

m_bn = get_batchnorm_model()

m_bn.compile(optimizer="rmsprop",
             loss="binary_crossentropy", 
             metrics=['accuracy'])

# we use only about half as many iterations since batch norm makes learning quicker
hist_bn = m_bn.fit_generator(train_generator,
                             epochs=int(0.5 * epochs),
                             steps_per_epoch=2000 // batch_size,
                             validation_data=validation_generator,
                             validation_steps=800 // batch_size)

NameError: ignored

In [ ]:
plot_histories([('batch_norm', hist_bn), 
                ('baseline', hist_base)], keys=['loss'])

##Using data augmentation
In order to make the most of our few training examples, we will "augment" them via a number of random transformations, so that our model would never see the exact same picture twice. This helps prevent overfitting and helps the model generalize better.

In [53]:
# now we need to set up a different input pipeline that does the augmentation for us
train_datagen_aug = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range=45,
    shear_range=0.2,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

train_generator_aug = train_datagen_aug.flow_from_directory(
        train_dir,  # this is the target directory
        target_size=(150, 150),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels

FileNotFoundError: ignored

In [54]:
# plot some augmented images: on every execution you should get different images
img_batch, label_batch = train_generator_aug.next()
n_per_row = 3

plt.figure(figsize=(15, 10))
for i in range(n_per_row):
    plt.subplot(2, n_per_row, i + 1)
    plt.imshow(img_batch[i])
    plt.title("class " + str(int(label_batch[i])))

for i in range(n_per_row):
    plt.subplot(2, n_per_row, n_per_row + i + 1)
    plt.imshow(img_batch[n_per_row + i])
    plt.title("class " + str(int(label_batch[n_per_row + i])))
    
plt.show()

NameError: ignored

In [ ]:
# during validation we do NOT augment the images of course
test_datagen_aug = ImageDataGenerator(rescale=1. / 255)
test_generator_aug = test_datagen_aug.flow_from_directory(
        validation_dir,  # this is the target directory
        target_size=(150, 150),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels

In [55]:
m_base = get_baseline_model()

m_base.compile(optimizer="rmsprop",
               loss="binary_crossentropy", 
               metrics=['accuracy'])

hist_aug = m_base.fit_generator(train_generator_aug,
                                epochs=epochs,
                                steps_per_epoch=2000 // batch_size,
                                validation_data=test_generator_aug,
                                validation_steps=800 // batch_size)

NameError: ignored

In [ ]:
plot_histories([('augmentation', hist_aug),
                ('baseline', hist_base)], keys=["loss"])